## Data and Objects - the Object-Relational-Mapping

We notice that when we find a correct relational model for our data, many of the rows are suggestive of exactly the data
we would expect to supply to an object constructor - data about an object. References to keys of other tables in rows suggest composition
relations while many-to-many join tables often represent aggregation relationships, and data about the relationship.

As a result of this, powerful tools exist to **automatically** create object structures from database schema, including saving and loading.

In [ ]:
import os
try:
    os.remove('molecules.db')
    print("Remove database to teach again from scratch")
except FileNotFoundError:
    print("No DB since this notebook was last run")

In [ ]:
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///molecules.db')

In [ ]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import relationship, declarative_base
Base = declarative_base()

class Element(Base):
    __tablename__ = "atoms"
    symbol = Column(String, primary_key=True)
    number = Column(Integer)
    molecules = relationship("AtomsPerMolecule", backref="atom")

In [ ]:
class Molecule(Base):
    __tablename__ = "molecules"
    name = Column(String, primary_key=True)
    mass = Column(Float)
    atoms = relationship("AtomsPerMolecule", backref="molecule")

In [ ]:
class AtomsPerMolecule(Base):
    __tablename__ = 'atoms_per_molecule'
    id = Column(Integer, primary_key=True)
    atom_id = Column(None, ForeignKey('atoms.symbol'))
    molecule_id = Column(None, ForeignKey('molecules.name'))
    number = Column(Integer)

If we now create our tables, the system will automatically create a DB:

In [ ]:
engine.echo=True

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
engine.echo=False

And we can create objects with a simple interface that looks just like ordinary classes:

In [ ]:
oxygen = Element(symbol='O',number=8)
hydrogen = Element(symbol='H', number=1)
elements = [oxygen, hydrogen]

In [ ]:
water = Molecule(name='water',mass=18.01)
oxygen_m = Molecule(name='oxygen', mass=16.00)
hydrogen_m = Molecule(name='hydrogen', mass=2.02)
molecules = [water, oxygen_m, hydrogen_m]

In [ ]:
amounts = [
    AtomsPerMolecule(atom=oxygen, molecule=water, number =1),
    AtomsPerMolecule(atom=hydrogen, molecule=water, number =2),
    AtomsPerMolecule(atom=oxygen, molecule=oxygen_m, number =2),
    AtomsPerMolecule(atom=hydrogen, molecule=hydrogen_m, number =2)
]

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session=Session()

In [ ]:
session.bulk_save_objects(elements+molecules+amounts)

In [ ]:
oxygen.molecules[0].molecule.name

In [ ]:
session.query(Molecule).all()[0].name

In [ ]:
session.commit()

This is a very powerful technique - we get our class-type interface in python, with database persistence and searchability for free!